In [1]:
import os
import pandas as pd
import json
import altair as alt

# Enable the dark theme
alt.themes.enable("dark")

path_to_json = '../data/'
# json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.startswith('steps-')]

dataframes = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        df = pd.json_normalize(json_text)
        df['dateTime'] = pd.to_datetime(df['dateTime'], format='%m/%d/%y %H:%M:%S')
        dataframes.append(df)

result = pd.concat(dataframes)
result.head(3)

,dateTime,value
0,2022-02-14 00:00:00,45
1,2022-02-14 00:01:00,10
2,2022-02-14 00:02:00,0


In [2]:
df['value'] = df['value'].astype(int)

In [3]:
df = df.groupby(df['dateTime'].dt.date).sum()
df = df.reset_index()

/var/folders/zg/b63jfq5j6pzb4_lby01nk6_m0000gn/T/ipykernel_22949/2663907322.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(df['dateTime'].dt.date).sum()


In [4]:
df['dateTime'] = pd.to_datetime(df['dateTime'])
# create a new column with the week number
df['week'] = df['dateTime'].dt.isocalendar().week
# group the data by week and calculate the mean for each week
df_weekly = df.groupby('week').mean().reset_index()

/var/folders/zg/b63jfq5j6pzb4_lby01nk6_m0000gn/T/ipykernel_22949/1017039611.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_weekly = df.groupby('week').mean().reset_index()


In [5]:
line_chart = alt.Chart(df).mark_line().encode(
    x='dateTime:T',
    y='value:Q'
).properties(
    title='Line Chart of My Data'
)

bar_chart = alt.Chart(df).mark_bar().encode(
    x='dateTime:T',
    y='value:Q'
).properties(
    title='Bar Chart of My Data'
)

line_chart_week = alt.Chart(df_weekly).mark_line().encode(
    x='week:N',
    y='value:Q'
).properties(
    title='Line Chart of My Data (Aggregated by Week)'
)

line_chart | bar_chart | line_chart_week

alt.HConcatChart(...)